In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
def request_page(url, year):
    conteudo = []

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    table = soup.find('table', {"class": "tablehead"})

    for tr in table.find_all('tr'):
        if 'RK' not in tr.text:
          linha = [year]
          for i, td in enumerate(tr.find_all('td')):
            if i == 1:
              a = td.find('a', href=True)
              detail = a['href'].split('/')
              playerId = detail[-2]
              playerSlug = detail[-1]
              linha.append(playerId)
              linha.append(playerSlug)
              linha.extend([ x.strip() for x in td.text.split(',')])
            else:
              linha.append(td.text)
          conteudo.append(linha)
    return conteudo


burl = url = 'https://www.espn.com/nba/salaries/_/year/{year}/page/{page}/seasontype/4'


In [ ]:

page = 1
year=1999
with open('./nba_salaries.csv', 'w') as  f:
    csv.delimiter = ';'
    writer = csv.writer(f)
    header = ['year', 'ranking', 'playerId', 'playerSlug','name', 'position', 'team', 'salary']
    writer.writerow(header)
    while year <= 2023:

        content = request_page(burl.format(year=year,page=page), year)

        while content:
          writer.writerows(content)
          page += 1
          content = request_page(burl.format(year=year,page=page), year)

        year += 1
        page = 1

